In [13]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 5

In [36]:
path = 'xab'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 7116320


In [37]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(f'unique chars: {len(chars)}')

unique chars: 28


In [38]:
SEQUENCE_LENGTH = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')

num training examples: 2372094


In [39]:
sentences[2]

'errun at  of the scedule rates.this is w'

In [40]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [41]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [43]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=5, shuffle=True).history
#history = model.fit(X, y, validation_split=0.05, batch_size=128)

Train on 2253489 samples, validate on 118605 samples
Epoch 1/5
2253489/2253489 [==============================] - 1386s 615us/step - loss: 1.5332 - accuracy: 0.5502 - val_loss: 1.6928 - val_accuracy: 0.5142
Epoch 2/5
2253489/2253489 [==============================] - 1394s 619us/step - loss: 1.5216 - accuracy: 0.5539 - val_loss: 1.7031 - val_accuracy: 0.5128
Epoch 3/5
2253489/2253489 [==============================] - 1387s 615us/step - loss: 1.5123 - accuracy: 0.5561 - val_loss: 1.6864 - val_accuracy: 0.5185
Epoch 4/5
2253489/2253489 [==============================] - 1388s 616us/step - loss: 1.5069 - accuracy: 0.5583 - val_loss: 1.6826 - val_accuracy: 0.5165
Epoch 5/5
2253489/2253489 [==============================] - 1396s 620us/step - loss: 1.5179 - accuracy: 0.5557 - val_loss: 1.7228 - val_accuracy: 0.5101


In [44]:
model.save('keras_model_small.h5')
pickle.dump(history, open("history_small.p", "wb"))

In [ ]:
model = load_model('keras_model.h5')
history = pickle.load(open("history.p", "rb"))

In [ ]:
def prepare_input(text):
    x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
        
    return x

In [ ]:
prepare_input("This is an example of input".lower())

In [ ]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion

In [ ]:
def predict_completions(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]

In [ ]:
sentences[200]

In [ ]:
quotes = [
    "this is what was",
    "written into the latest d",
    "i am reaching out",
    "can we schedule",
    "what do you"
]

In [ ]:
for q in quotes:
    seq = q[:40].lower()
    print(seq)
    print(predict_completions(seq, 5))
    print()